In [12]:
%load_ext autoreload
%autoreload 2

# Had issues pip installing psycopg2
# Run following command to resolve:
# env LDFLAGS="-I/usr/local/opt/openssl/include -L/usr/local/opt/openssl/lib" pip install psycopg2
# ref: https://stackoverflow.com/questions/39767810/cant-install-psycopg2-package-through-pip-install-is-this-because-of-sierra
import psycopg2
from psycopg2 import OperationalError

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def create_connection(
    db_name, db_user, db_password, 
    db_host, db_port):
    
    connection = None
    
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    
    return connection

def create_database(connection, db_name):
    query = f"CREATE DATABASE {db_name}"
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed succesfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [26]:
pw = input("Password:")
connection = create_connection(
    "medwatch", "postgres", pw, "127.0.0.1", "5432"
)

# create_database(connection, 'medwatch')


Password:
Connection to PostgreSQL DB successful


In [27]:
create_companies_table = """
CREATE TABLE IF NOT EXISTS companies (
  company TEXT,
  company_yahoo TEXT,
  symbol TEXT,
  exchange TEXT, 
  market_cap INTEGER,
  company_size TEXT,
  is_american BOOL,
  url_home TEXT,
  url_pr TEXT
)
"""

execute_query(connection, create_companies_table)

Query executed succesfully


In [28]:
entry = [
    ("AstraZeneca", "AstraZeneca PLC", "AZN", "NYSE", 139802, "large", True, "http://www.astrazeneca.com", "https://www.astrazeneca.com/media-centre/press-releases.html"),
    ("Sinovac", "Sinovac Biotech Ltd.", "SVA", "NASDAQ", 460249280, "small", True, "http://www.sinovacbio.com", "http://www.sinovacbio.com/?optionid=754")
]
entry_records = ", ".join(["%s"]*len(entry))

insert_query = (
    f"INSERT INTO  companies (company, company_yahoo, symbol, exchange, market_cap, company_size, is_american, url_home, url_pr) VALUES {entry_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, entry)